In [1]:
from typing import Any
import numpy as np
import torch
from DL import plot_graph
from Deep_cpu import MyModel
from Deep_cpu.activation import ReLU
from Deep_cpu.activation.loss import MeanSquaredError, CrossEntropyLoss
from Deep_cpu.optimizer.gdo import GradientDescentOptimizer

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

photo_nums = 64

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=photo_nums, shuffle=True)

loss_fn = CrossEntropyLoss()
op = GradientDescentOptimizer(lr=0.001, max_iterations=1)

layer_dict = {
    'first': {
        'type': 'linear',
        'in_features': 784,
        'out_features': 128,
        'activation': "ReLU"
    },
    'output': {
        'type': 'linear',
        'in_features': 128,
        'out_features': 10,
        'activation': "Softmax",
        'loss_fn': loss_fn
    }
}

model = MyModel(layers_dict=layer_dict, optimizer = op)
loss1 = []
# 数据预处理--One-Hot Encoding
k = 1
pre_x = []
pre_y = []

In [2]:
# m = 0
# a = []
# for batch in train_loader:
#     inputs, labels = batch
#     a.append(inputs[0][0])
#     m += 1
#     if m >= 6:
#         break

# plt.tight_layout()
# fig = plt.figure()
# plt.tight_layout()
# plt.imshow(a[0], cmap='gray', interpolation='none')
# plt.show()

In [23]:
epoch = 1
for j in range(epoch):
    running_loss = 0
    k=0
    for batch in train_loader:
        inputs, labels = batch

        x = inputs.numpy()
        y = labels.numpy()
        x = x.reshape(photo_nums, -1)

        y_one_hot = np.zeros((len(y), 10))
        for i in range(len(y)):
            y_one_hot[i, y[i]] = 1

        if j != 0:
            pre_x.append(x)
            pre_y.append(y)

        if x.shape[1] == 784:
            model.fit(x, y_one_hot)
            loss1.append(model.loss[-1])
            
        print(y)
        k += 1
        if k >= 1:
            break
    running_loss = model.loss[-1]
    print(f"Epoch {j + 1}, Loss: {running_loss}")

[0 9 3 8 0 9 6 2 2 6 8 7 7 0 8 1 7 7 4 2 2 1 8 2 0 3 0 2 7 1 3 0 8 8 1 7 5
 7 8 0 4 7 7 0 3 5 4 3 5 6 1 7 9 6 1 2 9 8 0 6 6 5 5 8]
Epoch 1, Loss: 3.729339641068198


In [37]:
result = model.predict(pre_x[0])
max_values = np.max(result, axis=0)
max_indices = np.argmax(result, axis=0)
k = max_indices
# for row, (max_value, max_index) in enumerate(zip(max_values, max_indices)):
#     print(f"Row {row}: Max Value = {max_value}, Index = {max_index}")
acc = np.sum(max_indices == k) / result.shape[1]
acc

1.0

In [5]:
pre_y[0]

array([6, 9, 5, 1, 9, 7, 2, 8, 2, 2, 4, 5, 8, 6, 1, 1, 4, 4, 3, 8, 4, 3,
       4, 3, 3, 3, 9, 0, 2, 4, 8, 9, 0, 9, 9, 5, 1, 8, 6, 2, 6, 9, 9, 3,
       3, 6, 8, 1, 6, 3, 4, 0, 2, 6, 0, 7, 3, 8, 3, 4, 1, 0, 5, 4],
      dtype=int64)

In [6]:
result[::, 0]

array([0.01605147, 0.01525115, 0.03981593, 0.00734688, 0.04530347,
       0.02809674, 0.80504431, 0.00352597, 0.0169963 , 0.02256779])

In [7]:
model.layers[-1].weight

array([[-0.11963784,  0.13610757,  0.02834925, ..., -0.07486002,
         0.18674346, -0.05949084],
       [-0.13707174,  0.13146469,  0.08422079, ...,  0.0556787 ,
        -0.11068311, -0.12555192],
       [ 0.08469717,  0.15547538,  0.17705488, ...,  0.0619268 ,
        -0.03965244,  0.10470472],
       ...,
       [ 0.09388816,  0.09626347,  0.03253431, ..., -0.18661029,
        -0.173889  ,  0.08246552],
       [ 0.09561025, -0.00129998, -0.17732991, ...,  0.11671903,
         0.23725235,  0.02322078],
       [ 0.06609013, -0.17575459, -0.18289767, ...,  0.04834118,
         0.04374584,  0.12030545]])

In [8]:
model.layers[-1].bias

array([[0.99856744, 0.99432405, 1.00630099, 1.00488167, 0.99962765,
        1.00185449, 0.99968328, 0.99827754, 1.00052393, 0.99595897]])